Alejandra Gavino-Dias González

# Práctica 8

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from ucimlrepo import fetch_ucirepo
import matplotlib.pyplot as plt

### Datos

In [2]:
optical_recognition_of_handwritten_digits = fetch_ucirepo(id=80) 
  
X = optical_recognition_of_handwritten_digits.data.features
y = optical_recognition_of_handwritten_digits.data.targets

X = MinMaxScaler().fit_transform(optical_recognition_of_handwritten_digits.data.features)
y = np.array(optical_recognition_of_handwritten_digits.data.targets).reshape(-1,)

Preparamos el dataset para hacer Holdout y probar el funcionamiento del objeto RLM

In [3]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=1./3., random_state=20, stratify=y)
print("Forma de X_train:", X_train.shape)
print("Forma de X_test:", X_test.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de y_test:", y_test.shape)

Forma de X_train: (3746, 64)
Forma de X_test: (1874, 64)
Forma de y_train: (3746,)
Forma de y_test: (1874,)


## Ejercicio 1

- **fit**: Método para ajustar el modelo a los datos de entrenamiento X e y. Utiliza OneHotEncoder para codificar la y en 0s y 1s. Luego, entrena un modelo de regresión lineal para cada clase utilizando los datos de características X y las etiquetas codificadas.
- **predict**: Predecir las etiquetas de clase para nuevos datos test. Utiliza los modelos entrenados previamente con la función fit para generar las predicciones de probabilidad para cada clase y devuelve un vector codificado con 1 en la clase predicha y 0 en las demás.
- **predict_proba**: Obbtenemos las probabilidades de predicción para cada clase utilizando la función softmax.
- **score**: evalua el rendimiento de la predicción que se ha hecho, comparando las predicciones con la clasificación y.

In [4]:
class rlm:
    
    def __init__(self):
        self.modelos = []
        self.n_clases = None
        self.predicciones = []
        
        
    def fit(self, X, y):
        self.n_clases = len(np.unique(y))
        deseada = OneHotEncoder(sparse_output = False).fit_transform(y.reshape(-1,1))
        #salida deseada de cada clasificador lineal (LinearRegression)
        for i in range(self.n_clases):
            self.modelos.append(LinearRegression())
            self.modelos[-1].fit(X, deseada[:,i])
    
    def predict(self, test):
        salidas = np.zeros((test.shape[0], self.n_clases)) #creamos una matriz salidas y la inicializamos con 0s
        for i, modelo in enumerate(self.modelos):
            salidas[:, i] = modelo.predict(test) #iteramos sobre cada modelo y almacenamos la prediccion en salidas
        self.predicciones = salidas
        vector_codificado = np.zeros_like(salidas) #esto es porque la salida tiene que ser un vector de 0s y un 1 en el maximo
        vector_codificado[np.arange(len(salidas)), np.argmax(salidas, axis=1)] = 1 #1 para la clase predicha
        
        return vector_codificado
       
    def predict_proba(self, pred):
        from scipy.special import softmax        
        salida = []
        for p in self.predicciones:
            salida.append(softmax(p))
        return salida
        
    def score(self, X, y):
        predicciones = np.argmax(self.predict(X), axis=1)  # Obtener las clases predichas
        return np.mean(predicciones == y)


### Comprobación usando Holdout

X_train, X_test, y_train e y_test están definidas arriba

In [5]:
modelo = rlm()

In [6]:
modelo.fit(X_train,y_train)

In [7]:
vector=modelo.predict(X_train)
print(vector) #solo para ver si es de la forma que queremos

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [8]:
probabilidades=modelo.predict_proba(vector)
print(probabilidades[0])

[0.09879228 0.0901471  0.08273895 0.10382833 0.08080132 0.19950696
 0.08662146 0.08201997 0.09881107 0.07673255]


In [9]:
scores=modelo.score(X_test,y_test)
print(scores)

0.935965848452508


## Ejercicio 2

Comparamos usando K-Fold con k=10 los métodos de regresión logística y regresión lineal múltiple. Obteniendo la tasa de aciertos al final para cada fold con 10000 iteraciones y una tasa de aciertos media para cada método para comparar.

In [10]:
c = StratifiedKFold(n_splits = 10, shuffle=True)
rlm_lista = []
rlog = []

In [11]:
for i_train, j_test in c.split(X, y):
    X_train = X[i_train]
    X_test = X[j_test]
    y_train = y[i_train].ravel() #ravel: Return a contiguous flattened array (numpy)
    y_test = y[j_test].ravel()
    modelo = rlm()
    modelo.fit(X_train, y_train)
    rlm_lista.append(modelo.score(X_test, y_test))
    
    l = LogisticRegression(max_iter=10000)
    l.fit(X_train, y_train)
    rlog.append(l.score(X_test,y_test))
    
    print("Tasa de aciertos: RLM=",rlm_lista[-1],"\tRLOG=",rlog[-1])
print("\nTasa de aciertos media: ")
print("RML: ", np.mean(np.array(rlm_lista)))
print("RLOG: ",np.mean(np.array(rlog)))

Tasa de aciertos: RLM= 0.9217081850533808 	RLOG= 0.9608540925266904
Tasa de aciertos: RLM= 0.9252669039145908 	RLOG= 0.9626334519572953
Tasa de aciertos: RLM= 0.9341637010676157 	RLOG= 0.9715302491103203
Tasa de aciertos: RLM= 0.9466192170818505 	RLOG= 0.9679715302491103
Tasa de aciertos: RLM= 0.9395017793594306 	RLOG= 0.9750889679715302
Tasa de aciertos: RLM= 0.9377224199288257 	RLOG= 0.9804270462633452
Tasa de aciertos: RLM= 0.9359430604982206 	RLOG= 0.9768683274021353
Tasa de aciertos: RLM= 0.9359430604982206 	RLOG= 0.9679715302491103
Tasa de aciertos: RLM= 0.9341637010676157 	RLOG= 0.9715302491103203
Tasa de aciertos: RLM= 0.9306049822064056 	RLOG= 0.9697508896797153

Tasa de aciertos media: 
RML:  0.9341637010676157
RLOG:  0.9704626334519573
